## Using XGBoost to Predict stock market prices given a companies summary statistics

In [1]:
from market import *
from market_ml import *

C:\Users\kevin\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
C:\Users\kevin\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\kevin\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be remov

In [2]:
update_csv()

Getting data for: PIH
Getting data for: TURN
Getting data for: FLWS
Getting data for: FCCY
Getting data for: SRCE
Getting data for: VNET
Getting data for: TWOU
Getting data for: TPNL
Failed to parse json response
Ticker: TPNL did not work.
Getting data for: JOBS
Getting data for: EGHT
Getting data for: AAON
Getting data for: ABEO
Getting data for: ABIL
Getting data for: ABMD
Getting data for: AXAS
Getting data for: ACIU
Getting data for: ACIA
Getting data for: ACTG
Getting data for: ACHC
Getting data for: ACAD
Getting data for: ACST
Getting data for: AXDX
Getting data for: XLRN
Getting data for: ARAY
Getting data for: ACRX
Getting data for: ACER
Getting data for: ACET
Failed to parse json response
Ticker: ACET did not work.
Getting data for: AKAO
Failed to parse json response
Ticker: AKAO did not work.
Getting data for: ACHV
Getting data for: ACHN
Getting data for: ACIW
Getting data for: ACRS
Getting data for: ACMR
Getting data for: ACOR
Getting data for: ATVI
Getting data for: ADMS
Ge

In [6]:
model = train_and_get_model()
predict_price('FB')

Training XGB model with hyperparameter tuning... Make sure csv is updated.
Test Score: 0.9268368922704839
RMSE: 13.977278
Using last saved model.


186.33849

In [115]:
portfolio = ['EIX', 'RCUS', 'DIS', 'ATVI', 'A', 'EA', 'IIPR']
predictions = []
for ticker in portfolio:
    predictions.append(predict_price(ticker))
print(predictions)

[49.56896, 11.598441, 157.96178, 46.519207, 62.09287, 85.7837, 76.2089]


In [147]:
# save model to file
pkl.dump(model, open("xgbr_0725.dat", "wb"))

In [ ]:
model = pickle.load(open("xgbr_0725.dat", "rb"))

## Using a Neural Net to Predict Prices 

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=123)
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(20, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5)
model.evaluate(X_test, y_test)

Epoch 1/5
1780/1780 [==============================] - 0s 181us/step - loss: nan - acc: 0.0000e+00
Epoch 2/5
1780/1780 [==============================] - 0s 29us/step - loss: nan - acc: 0.0000e+00
Epoch 3/5
1780/1780 [==============================] - 0s 31us/step - loss: nan - acc: 0.0000e+00
Epoch 4/5
1780/1780 [==============================] - 0s 31us/step - loss: nan - acc: 0.0000e+00
Epoch 5/5
446/446 [==============================] - 0s 190us/step


[nan, 0.0]

In [7]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import max_norm

def create_nnet_model():
    activation='relu'
    dropout_rate=0.0
    init_mode='uniform'
    weight_constraint=0
    optimizer='adam'
    momentum=0
    
    # Create Model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(20, activation=tf.nn.relu,
                             kernel_initializer=init_mode,
                             kernel_constraint=max_norm(weight_constraint)),
        tf.keras.layers.Dense(20, activation=tf.nn.relu,
                             kernel_initializer=init_mode,
                             kernel_constraint=max_norm(weight_constraint)),
        tf.keras.layers.Dense(20, activation=tf.nn.relu,
                             kernel_initializer=init_mode,
                             kernel_constraint=max_norm(weight_constraint)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(1, kernel_initializer=init_mode, activation='sigmoid')
    ])
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

print('Training Neural Net with hyperparameter tuning... Make sure csv is updated.')
financial_data = pd.read_csv("company_statistics.csv")
to_remove = ['Ticker', 'Name', 'Price', 'Sector', 'Industry', 'IPO Year']
feature_cols = [x for x in financial_data.columns if x not in to_remove]
data_cleaned = financial_data.fillna(-1)
X = financial_data[feature_cols]
Y = financial_data['Price']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=123)
    
print("Right before defining KerasClassifier Model.")
model = KerasClassifier(build_fn=create_nnet_model, batch_size=1000, epochs=10, verbose=1)    

param_grid = {
    'epochs': [10,  20,  50],
    'batch_size': [128, 1000, 5000]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=1)
print("Right before fitting the grid.")
grid_result = grid.fit(X_train.values, y_train.values)

print("Best %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Training Neural Net with hyperparameter tuning... Make sure csv is updated.
Right before defining KerasClassifier Model.
Right before fitting the grid.
Fitting 3 folds for each of 9 candidates, totalling 27 fits


ValueError: clipnorm is not a legal parameter

In [1]:
predict_price("NVDA", model=grid_result.best_estimator_, model_type='net')

NameError: name 'predict_price' is not defined

In [37]:
%matplotlib inline
import pathlib
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 8, 6
import pandas as pd
import numpy as np
import seaborn as sns
pd.set_option('display.max_columns', 500)
from collections import defaultdict

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

pd.options.mode.chained_assignment = None

from torch.nn import init
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils import data
from torch.optim import lr_scheduler

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

from tqdm import tqdm, tqdm_notebook, tnrange
tqdm.pandas(desc='Progress')

In [42]:
from torch.nn import init
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils import data
from torch.optim import lr_scheduler 

class RegressionColumnarDataset(data.Dataset):
    def __init__(self, df, cats, y):
        self.dfcats = df[cats]
        self.dfconts = df.drop(cats, axis=1)
        
        #elf.cats = np.stack([c.values for n, c in self.dfcats.items()], axis=1).astype(np.int64)
        self.conts = np.stack([c.values for n, c in self.dfconts.items()], axis=1).astype(np.float32)
        self.y = y.values.astype(np.float32)
        
    def __len__(self): return len(self.y)
 
    def __getitem__(self, idx):
        return [self.cats[idx], self.conts[idx], self.y[idx]]
 

financial_data = pd.read_csv("company_stats_2019-08-30.csv")
to_remove = ['Ticker', 'Name', 'Price', 'IPO Year', 'Sector', 'Industry']
categorical = []#ector', 'Industry']
feature_cols = [x for x in financial_data.columns if x not in to_remove]
X = financial_data[feature_cols]
Y = financial_data['Price']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=123)
    
trainds = RegressionColumnarDataset(X_train, categorical, y_train)
valds = RegressionColumnarDataset(X_test, categorical, y_test)
 
params = {'batch_size': 128,
          'shuffle': True,
          'num_workers': 0}
 
traindl = data.DataLoader(trainds, **params)
valdl = data.DataLoader(valds, **params)

In [43]:
class MixedInputModel(nn.Module):
    def __init__(self, n_cont, out_sz, szs, drops, y_range, use_bn=True):
        super().__init__()
        #for i,(c,s) in enumerate(emb_szs): assert c > 1, "cardinality must be >=2, got emb_szs[{i}]: ({c},{s})"
        #self.embs = nn.ModuleList([nn.Embedding(c, s) for c,s in emb_szs])
        #for emb in self.embs: emb_init(emb)
        #n_emb = sum(e.embedding_dim for e in self.embs)
        #self.n_emb, self.n_cont=n_emb, n_cont
        
        szs = [n_cont] + szs
        self.lins = nn.ModuleList([nn.Linear(szs[i], szs[i+1]) for i in range(len(szs)-1)])
        self.bns = nn.ModuleList([nn.BatchNorm1d(sz) for sz in szs[1:]])
        for o in self.lins: nn.init.kaiming_normal_(o.weight.data)
        self.outp = nn.Linear(szs[-1], out_sz)
        nn.init.kaiming_normal_(self.outp.weight.data)

        #self.emb_drop = nn.Dropout(emb_drop)
        self.drops = nn.ModuleList([nn.Dropout(drop) for drop in drops])
        self.bn = nn.BatchNorm1d(n_cont)
        self.use_bn,self.y_range = use_bn,y_range

    def forward(self, x_cat, x_cont):
        if self.n_emb != 0:
            x = [e(x_cat[:,i]) for i,e in enumerate(self.embs)]
            x = torch.cat(x, 1)
            x = self.emb_drop(x)
        if self.n_cont != 0:
            x2 = self.bn(x_cont)
            x = torch.cat([x, x2], 1) if self.n_emb != 0 else x2
        for l,d,b in zip(self.lins, self.drops, self.bns):
            x = F.relu(l(x))
            if self.use_bn: x = b(x)
            x = d(x)
        x = self.outp(x)
        if self.y_range:
            x = torch.sigmoid(x)
            x = x*(self.y_range[1] - self.y_range[0])
            x = x+self.y_range[0]
        return x.squeeze()

y_range = (0, y_train.max()*1.2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using: " + str(device))
from tqdm import tqdm, tqdm_notebook, tnrange
tqdm.pandas(desc='Progress')

m = MixedInputModel(n_cont=len(financial_data.columns)-len(categorical),  
                    out_sz=1, 
                    szs=[1000,500,250], 
                    drops=[0.001,0.01,0.01], 
                    y_range=y_range).to(device)


Using: cpu


In [45]:
def fit(model, train_dl, val_dl, loss_fn, opt, scheduler, epochs=3):
    num_batch = len(train_dl)
    for epoch in tnrange(epochs):      
        y_true_train = list()
        y_pred_train = list()
        total_loss_train = 0          
        t = tqdm_notebook(iter(train_dl), leave=False, total=num_batch)
        for cat, cont, y in t:
            cat = cat.cuda()
            cont = cont.cuda()
            y = y.cuda()
            
            t.set_description('Epoch {epoch}')
            
            opt.zero_grad()
            pred = model(cat, cont)
            loss = loss_fn(pred)#, y)
            loss.backward()
            lr[epoch].append(opt.param_groups[0]['lr'])
            tloss[epoch].append(loss.item())
            scheduler.step()
            opt.step()
            
            t.set_postfix(loss=loss.item())
            
            y_true_train += list(y.cpu().data.numpy())
            y_pred_train += list(pred.cpu().data.numpy())
            total_loss_train += loss.item()
            
        train_acc = rmse(y_true_train, y_pred_train)
        train_loss = total_loss_train/len(train_dl)
        
        if val_dl:
            y_true_val = list()
            y_pred_val = list()
            total_loss_val = 0
            for cat, cont, y in tqdm_notebook(val_dl, leave=False):
                cat = cat.cuda()
                cont = cont.cuda()
                y = y.cuda()
                pred = model(cat, cont)
                loss = loss_fn(pred, y)
                
                y_true_val += list(y.cpu().data.numpy())
                y_pred_val += list(pred.cpu().data.numpy())
                total_loss_val += loss.item()
                vloss[epoch].append(loss.item())
            valacc = rmse(y_true_val, y_pred_val)
            valloss = total_loss_val/len(valdl)
            print('Epoch {epoch}: train_loss: {train_loss:.4f} train_rmse: {train_acc:.4f} | val_loss: {valloss:.4f} val_rmse: {valacc:.4f}')
        else:
            print('Epoch {epoch}: train_loss: {train_loss:.4f} train_rmse: {train_acc:.4f}')
    
    return lr, tloss, vloss
 
opt = optim.Adam(m.parameters(), 1e-2)
lr_cosine = lr_scheduler.CosineAnnealingLR(opt, 1000)
 
lr = defaultdict(list)
tloss = defaultdict(list)
vloss = defaultdict(list)
 
lr, tloss, vloss = fit(model=m, train_dl=traindl, val_dl=valdl, loss_fn=F.mse_loss, opt=opt, scheduler=lr_cosine, epochs=10)

AttributeError: 'RegressionColumnarDataset' object has no attribute 'cats'